# Data processing

In this notebook, I am loading a list of molecules I obtained from ChEMBL and processing them to make sure I have:
- Standard SMILES representation of the compound
- InChIKey associated to the compound

In [ ]:
# In this codeblock I will import the necessary packages and specify the paths to relevant folders

In [1]:
# In this codeblock I will load the data from the /data folder to a Pandas dataframe and understand which headers it has

In [ ]:
# In this codeblock I will convert the molecules to standard SMILES by using the function standardise_smiles from /src
# I will import the function directly from src, not copying it here

In [ ]:
# In this codeblock I will get the Inchikey representation of the molecules using the RDKIT package

In [2]:
# In this codeblock I will save the data as a .csv file containing only the standard smiles and the inchikey as columns. 
# All data will be saved with informative names in the /data folder

# Model Bias Evaluation

Now, I will use the predictions I got from the Ersilia Model Hub on the dataset of 1000 molecules curated above and see how are they distributed in their space (which might be 0 to 1 for probabilities, or different for regression models)

In [1]:
# In this codeblock I will load the predictions I've run on Ersilia and saved in the /data folder

In [ ]:
# In this codeblock I will create the necessary plots with MatPlotLib to observe the distribution of predicted values